In [ ]:
# years = pd.DataFrame({'year':pd.date_range(start='1998', end='2022',freq='Y')})
# years =(years.assign(year=years.year.dt.year.astype(str)))
# years.info()
# years_in_scope = years.year.values
# years_in_scope

# active_cik = df.query("rdate.dt.year.astype('string').isin(@years_in_scope)")['cik'].unique()

In [ ]:
%%time

from pathlib import Path
import pandas as pd
import pyarrow as pa

# my_file = Path(r"E:\app_data\sec_apps_data\speed_test\filings_13f_full\tr_04_filings_parquet_cik")
# columns = ['cik', 'accession_number', 'value', 'rdate', 'quarter']
# df = pd.concat(pd.read_parquet(f, columns=columns, filters = [('tr_type',  '!=' , pa.scalar("CLOSE"))]) for f in my_file.glob("*.parquet"))

df = df.assign(
                 # cik=df.cik.astype(str),
               ryear=df.rdate.dt.year.astype('category'),
#                value_len=df.value.astype(str).str.len()-2
               )

In [ ]:
df.info()
# df.query("cik==14661").head(50)


In [ ]:
#  find ciks that have been active in 1999 and 2020
df_cik_year = df[['cik', 'ryear']].drop_duplicates()
df_cik_year = df_cik_year.sort_values(by=['cik', 'ryear'],ascending=[True,False])

df_cik_year_1999_2020 = df_cik_year.query("ryear.astype('str') == '2020' | ryear.astype('str') == '1999'")

cik_1999_2020 = (df_cik_year_1999_2020
           .groupby(by=['cik']).count()
           .reset_index()
           .query("ryear == 2 ")
           .drop_duplicates()['cik'].to_list()
           )

# cik_1999_2020     # 1067983 berkshire
len(cik_1999_2020) # 647

In [ ]:

# np.array(cik_1999_2020)

for element in cik_1999_2020:
    print(element, end=',')


In [ ]:
# len(cik_1999_2020)
# print(cik_1999_2020[:20])
# [2230, 3520, 5272, 7195, 7789, 9015, 10742, 14661, 16972, 18349, 18748, 19475, 19617, 21175, 22657, 24386, 35442, 35527, 36066, 36104]
# (df.query("cik.isin(@cik_1999_2020)"))[['cik']].drop_duplicates().sort_values(by=['cik']).shape

In [ ]:
# limit the big df to a subset where only the cik active from 1999 to 2020 are present
# df.head()
sub_df = (df
          .query("cik.isin(@cik_1999_2020)")
          .groupby(by=['cik','quarter'])
          .agg({'value':'sum', 'accession_number':'first'})
          .reset_index()[['cik','quarter', 'value', 'accession_number']])


In [2]:
# working with slider / selector
import altair as alt
from altair import datum

input_dropdown = alt.binding_select(options=cik_1999_2020, name='cik')
selection = alt.selection_single(fields=['cik'], bind=input_dropdown)



alt.Chart(sub_df).mark_point().transform_calculate(value_billion="datum.value / 1000000000")\
    .encode(
    alt.X('quarter:O'),
    alt.Y('value'),
    # alt.Row('cik'),
    tooltip=['quarter:O','accession_number:N', 
             alt.Tooltip('value:Q', format="$.3s") ,
             alt.Tooltip('value_billion:Q', format="$.3", title='In Billions'),
             alt.Tooltip('value:Q', format="$~s")
            ]
    # alt.Tooltip(['quarter:O','accession_number:N', 'value:O']),
    
).add_selection(selection).transform_filter(selection).interactive().properties(width=800, height=200)

NameError: name 'cik_1999_2020' is not defined

In [ ]:
from vega_datasets import data
import pandas as pd

# source = data.cars()

# alt.Chart(source).mark_circle(size=60).encode(
#     x='Horsepower',
#     y='Miles_per_Gallon',
#     color='Origin',
#     tooltip=[
#         'Origin',
#         alt.Tooltip('Horsepower', format="$.3s", title="Valor")
#     ]
# )

data = pd.DataFrame({
    'x': [1, 2, 3, 4, 5, 6],
    'y': [100000000000, 200000000000000, 300000000, 400000000, 500000000, 300000]
})

tooltip = [alt.Tooltip('x', format='0.0f'), alt.Tooltip('y', format='$~s')] 



chart = alt.Chart(data).mark_point().encode(
    x=alt.X('x:Q'),
    y=alt.Y('y:Q', scale=alt.Scale(type='log')),
    tooltip=tooltip,

)

chart.display()

### My 676 main `cik` dataset to learn with 

In [ ]:
%%time
df_small = df.query("cik.astype('int').isin(@cik_1999_2020)")
df_small = df_small.query("cik.astype('int').isin(@cik_1999_2020)")

df_small = (df_small.groupby(by=['cik', 'quarter']).agg({'accession_number':'last',
                                                         'value':'sum',
                                                         'rdate':'last',
                                                         'ryear':'last',
                                                         'value_len':'max'}))

df_small = df_small.reset_index()
df_small = df_small.query("cik.astype('int').isin(@cik_1999_2020)")

df_small_year = (df_small.groupby(by=['cik', 'ryear']).agg({'value':'sum',
                                                            'value_len':'max'}))

df_small_year = df_small_year.reset_index()
df_small_year = df_small_year.query("cik.astype('int').isin(@cik_1999_2020)")

df = None

In [ ]:
# df_small.info()
# df_small.head(100)
# df_small.cik.nunique()
# df_small_year.head(10)
df_small_year.cik.nunique()

In [ ]:
# df_small.query("cik.astype('int').isin(@cik_1999_2020)")
# df_small.head(20)
df_small_year.head(200)

### Jake VanderPlas - Altair - PyCon 2018

In [ ]:
pd.set_option('display.max_columns', 35)
pd.set_option("display.max_colwidth",30)
pd.set_option("display.max_rows", 500)


import pandas as pd
from vega_datasets import data
import altair as alt
alt.data_transformers.disable_max_rows()

In [ ]:
cars = data.cars()

cars.head(2)

In [ ]:
alt.Chart(cars).mark_circle().encode(
    x='Miles_per_Gallon',
    y='Horsepower'
).interactive().to_dict()

In [ ]:
cars1 = cars.iloc[:1]

spec = alt.Chart(cars).mark_circle().encode(
    x='Miles_per_Gallon',
    y='Horsepower'
).interactive().to_dict()

In [ ]:
alt.Chart.from_dict(spec).encode(color='Origin:N')

In [ ]:
alt.Chart(cars).mark_line().encode(
    x='Miles_per_Gallon',
    y='Horsepower'
).interactive()

In [ ]:
cik_1999_2020[:10]

In [ ]:

alt.Chart(df_small_year[df_small_year.cik==5272][['ryear', 'value']]).mark_bar().encode(
    x='ryear:O',
    y='value'
).interactive()



In [ ]:
alt.Chart(cars).mark_bar().encode(
    x='Miles_per_Gallon',
    y='Horsepower'
).interactive()

In [ ]:
alt.Chart(cars).mark_bar().encode(
    x=alt.X('Miles_per_Gallon', bin=True),
    y='count()'
    
).interactive()

In [ ]:
%pip install autoviz